### Notebook for preparing the samples

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")

In [3]:
import pandas as pd
import numpy as np

from plotnine import *

In [4]:
defaults = pd.read_csv("./input/submission/defaults__submission.csv")

In [5]:
defaults.loc[:, 'report_id'] = defaults.loc[:, 'report_id'].astype(str)

In [7]:
from utils.data_preparation import preprocess_lookups

Preprocessing the lookups is expensive, and has been done before-hand. Uncomment to run

In [8]:
#preprocess_lookups(nrows=100000)

In [9]:
lookups = pd.read_pickle("./input/submission/lookups_converted_fields__submission.df")

In [10]:
print("""
Lookups: {lrows}, Variables: {lcols}
Defaults: {drows}, Variables: {dcols} 
""".format(lrows = lookups.shape[0], lcols = lookups.shape[1],
           drows = defaults.shape[0], dcols = defaults.shape[1]))


Lookups: 45995, Variables: 51
Defaults: 4819, Variables: 3 



Join with defaults

In [11]:
mdf = defaults.join(lookups, rsuffix="_r").drop(columns=['report_id', 'report_id_r'])

Drop the loans that are missing lookup information

In [12]:
mdf.dropna(inplace=True)

In [13]:
print("""
Joined dataframe, Rows: {rows}, Cols: {cols}
""".format(rows=mdf.shape[0], cols=mdf.shape[1]))


Joined dataframe, Rows: 4061, Cols: 52



Unstack list columns so that we get one row per statement year

In [14]:
df_long = mdf.set_index(['loan_id', 'default_at_9']).apply(pd.Series.explode).reset_index()

Use only statements that covers 12 months

In [15]:
df_long = df_long.loc[df_long['months_covered'] == 12].drop('months_covered', axis=1)

Setup variables that are fractions and not

In [16]:
from utils.data_preparation import fetch_fs_vars 

In [17]:
fs_vars = fetch_fs_vars()
pct_vars =  []
raw_vars = pd.Series(fs_vars)[~pd.Series(fs_vars).isin(pct_vars)].to_list()

Subset the vars plus the loan_id

In [18]:
df_raw = df_long[['loan_id'] + raw_vars]

In [19]:
from utils.data_preparation import prepare_diffed

Prepare three datasets:
* One with the raw values statement values (already given by `df_long` if removing target),
* One with the difference between the statement values, and 
* One with the sign of the difference between statement values.

In [20]:
target = df_long[["default_at_9", "loan_id"]].set_index("loan_id")
target.to_csv("./input/submission/target__submission.csv")

In [21]:
df_raw_vals = df_long.drop("default_at_9", axis=1)
df_diff = prepare_diffed(df_raw, df_long, pct_vars, raw_vars, sign=False)
df_diff_sign = prepare_diffed(df_raw, df_long, pct_vars, raw_vars, sign=True)

Convert all datasets to wide format, with previous statement values given as features. This computation is quite expensive, so the wide format data frames are prepared and then saved to disk.

In [140]:
from utils.data_preparation import long_to_wide
from tqdm import tqdm

In [146]:
def wide_and_reset(df_base, features):
    """Util to make clean."""
    df_wide = df_base.groupby('loan_id').apply(long_to_wide, features=features)
    df_wide.reset_index(level=0, drop=True, inplace=True)
    return df_wide.dropna(axis=0)

In [148]:
dfs = [df_raw_vals, df_diff, df_diff_sign]
paths = ["df_wide_raw_vals.df", "df_wide_diff.df", "df_wide_diff_sign.df"]
for df, path in tqdm(zip(dfs, paths)):
    ## Pickle, instead of .to_csv to keep dtypes
    X = wide_and_reset(df, features = raw_vars + pct_vars)
    X.to_pickle("./input/submission/" + path)

3it [04:25, 88.52s/it]
